# 2.3 Создание и обработка новых ID авторов

На этом этапе создаются новые авторские идентификаторы на основании сокращенных данных ID, фамилии, инициалов и ID аффилиаций.

Этап 1: Создание новых ID
- Шаг 1: Создание новых ID. Создаются новые колонки для создаваемых ID. К цифровому ID присуждаются данные фамилии, инициалов и ID аффилиации для упрощения задачи дедупликации авторов.
- Шаг 2: Оценка близости новых ID. При помощи метрики Дамерау-Левенштайна рассчитывается близость между уникальными новыми ID. В случае, если строки расходятся менее чем на 5 символов, создается пара-кандидат на мэтчинг.

Этап 2: Обработка новых ID
- Шаг 0: создание базовых объектов. Технический шаг, на котором создается датафрейм для последующей обработки пар-кандадитов на мэтчинг.
- Шаг 1: обработка авторов с наличествующим цифровым ID. Итеративно фиксируем по одному признаку нового ID (фамилия, инициалы, аффилиации), исследуем паттерны различий между обнаруженными строками и адаптируем строки. (Note: этот шаг проще следующего, поскольку технически авторы с заданным цифровым ID уже идентифицированы, а наша работа заключается в нормализации написания их данных)
- Шаг 2: обработка авторов с отсутствующим идентификатором. Аналогично итеративно проходимся по параметрам различий между новыми ID, однако объединяем строки более консервативно ввиду отсутствия цифрового ID.
- Шаг 3: финализация обработки. Создаем финальные ID. На этом этапе также объединяются в единый список множественные аффилиации одного автора, определенные в ходе дедупликации новых ID.
- Шаг 4: изменение рабочего файла с данными. Колонки с новыми ID адаптируются в соответствии с результатами обработки.

In [ ]:
import requests
from urllib.parse import urlencode
import pandas as pd
import urllib
import json
from tqdm import tqdm
import re

In [ ]:
folder_url = 'YANDEX DISK FOLDER LINK' #ссылка на папку
file_url = 'FILE NAME' #тут должно быть имя файла
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download' + '?public_key=' + urllib.parse.quote(folder_url) + '&path=/' + urllib.parse.quote(file_url)

r = requests.get(url)
h = json.loads(r.text)['href']
df = pd.read_excel(h)

# Создание новых ID

## Шаг 1: Создание новых ID

In [ ]:
def create_id(df, num):
  '''
  Эта функция принимает на вход датафрейм и номер (можно также заменить на название колонки)
  Чтобы функция работала с любым датафреймом, достаточно заменить названия колонок в [] на соответствующие по смыслу в другой таблице
  Оригинальные названия сохранены для понимания принципа работы функции и логики замены

  df - pandas DataFrame
  num - количество повторений (количество нумерованных колонок с id, максимальное количество авторов в базе)

  Функция проводит изменения в оригинальном датафрейме
  Если вы хотите оставить его неизменным и сохранить измененный df в новую переменную, стоит разкомментировать выход функции в конце
  '''
  for i in range(1, num+1):
    inds = []
    for i in tqdm(range(len(df))):
        if df['author_{}_eng last name'.format(num)][i] != 'none':
            try:
                surname = df['author_{}_eng last name'.format(num)][i][:8]
            except len(surname) < 8:
                surname = df['author_{}_eng last name'.format(num)][i]
        elif df['author_{}_eng last name'.format(num)][i] == 'none' and df['author_{}_ru last name'.format(num)][i] != 'none':
            surname = translit(df['author_{}_ru last name'][i],
                               'ru',
                               reversed = True)
        else:
            ind = 'N/A'
            inds.append(ind)
            continue


        ind = '_'.join([str(df['author_{}_id'.format(num)][i]),
                        df['author_{}_eng initials'.format(num)][i].replace('.',''),
                        surname,
                        str(df['author_{}_affiliation_id'.format(num)][i][0]),
                      ])
        inds.append(ind)
    df['new_id_{}'.format(num)] = inds
  #return df

In [ ]:
create_id(df, 9)

# объединяем все новые id в список уникальных значений
all_new_ids = []
for i in range(1,9):
    all_new_ids += cop_df['new_id_{}'.format(i)].to_list()
all_new_ids = list(set(all_new_ids))
all_new_ids.remove('N/A') #удаляем филлерное значение из списка уникальных значений id

## Шаг 2: Оценка близости новых ID

In [ ]:
import jellyfish

In [ ]:
distances = {}
for i in tqdm(all_new_ids):
    for j in all_new_ids:
        a = jellyfish.damerau_levenshtein_distance(i,j)
        if 0 < a <= 4: #рассматриваем только те пары, где разница между строками <= 4 символам
            distances[(i,j)] = a

In [ ]:
sort_dist = dict(sorted(distances.items(), key=lambda item: item[1]))

# в словаре ниже получаем пары строк и разницу между ними
dist_to_sort = pd.DataFrame()
dist_to_sort['vals'] = distances.keys()
dist_to_sort['diff'] = distances.values()
val1 = []
val2 = []
for i in range(len(dist_to_sort['vals'])):
    dist_to_sort['vals'][i] = sorted(list(dist_to_sort['vals'][i]))
    val1.append(dist_to_sort['vals'][i][0])
    val2.append(dist_to_sort['vals'][i][1])
dist_to_sort['val1'] = val1
dist_to_sort['val2'] = val2

Теперь на основании датафрейма **dist_to_sort** следует принять решение об объединении подобранных пар-кандидатов на мэтчинг.

In [ ]:
dist_to_sort.to_excel('mismatch.xlsx')

# Обработка новых ID

## Шаг 0: Создание базовых объектов

In [ ]:
df = pd.read_excel('mismatch.xlsx').drop('Unnamed: 0', axis = 1)
# файл содержит данные о попарных сравнениях значений, отфильтрованные по критерию diff (расстояние) <= 4
df['id1'] = pd.Series()
df['id2']= pd.Series()
df['init1']= pd.Series()
df['init2']= pd.Series()
df['surname1']= pd.Series()
df['surname2']= pd.Series()
df['aff1']= pd.Series()
df['aff2']= pd.Series()

# трансформируем датафрейм так, чтобы были отдельно доступны все составные части ID: идентификатор, инициалы, сокращенная фамилия, аффилиации
for i in tqdm(range(len(df))):
    vals1 = df['val1'][i].split('_')
    vals2 = df['val2'][i].split('_')
    df['id1'][i] = vals1[0]
    df['id2'][i] = vals2[0]
    df['init1'][i] = vals1[1]
    df['init2'][i] = vals2[1]
    df['surname1'][i]= vals1[2]
    df['surname2'][i]= vals2[2]
    df['aff1'][i]= vals1[3]
    df['aff2'][i]= vals2[3]

100%|██████████| 26018/26018 [00:03<00:00, 8603.82it/s]


In [ ]:
df.head()

,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2
0,1,0_EA_Kozhevni_none,0_IA_Kozhevni_none,0,0,EA,IA,Kozhevni,Kozhevni,none,none
1,1,483923_OM_Shtompel_322,483923_O_Shtompel_322,483923,483923,OM,O,Shtompel,Shtompel,322,322
2,1,118648_GI_Osadchaj_1432,118648_G_Osadchaj_1432,118648,118648,GI,G,Osadchaj,Osadchaj,1432,1432
3,1,514063_JUP_Lezhnina_421,514063_JU_Lezhnina_421,514063,514063,JUP,JU,Lezhnina,Lezhnina,421,421
4,1,0_VG_Gojdenko_18476,0_V_Gojdenko_18476,0,0,VG,V,Gojdenko,Gojdenko,18476,18476


In [ ]:
df['status'] = ['issue']*len(df) # добавляем колонку статуса: по дефолту все кейсы неразрешенные

In [ ]:
end_dic = {} #здесь ключами будут готовые избранные ID, а значениями - все варианты написания, которые мы унифицируем этим ID

In [ ]:
def check_done(df):
  for i in tqdm(range(len(df))):
    for k in end_dic.values():
        if df.val1[i] in k:
            df['status'][i] = 'done'
  print(len(df[df['status'] == 'done']))

In [ ]:
check_done(df)

100%|██████████| 26018/26018 [00:00<00:00, 1909357.20it/s]

0


## Шаг 1: Обработка авторов с наличествующим идентификатором

In [ ]:
# для удобства создаем суб-таблицу, содержащую только тех авторов, у которых в id первой строки не указан 0
elid = df[df['id1'] != '0'].reset_index()
elid['end_res'] = pd.Series()

In [ ]:
# замечаем, что и у авторов с заданными ID есть различия в новых ID в виду неоднородного написания инициалов, фамилий и указания аффилиаций
elid.head()

,index,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
0,1,1,483923_OM_Shtompel_322,483923_O_Shtompel_322,483923,483923,OM,O,Shtompel,Shtompel,322,322,issue,NaN
1,2,1,118648_GI_Osadchaj_1432,118648_G_Osadchaj_1432,118648,118648,GI,G,Osadchaj,Osadchaj,1432,1432,issue,NaN
2,3,1,514063_JUP_Lezhnina_421,514063_JU_Lezhnina_421,514063,514063,JUP,JU,Lezhnina,Lezhnina,421,421,issue,NaN
3,5,1,761559_VN_Kazantse_2602,761559_V_Kazantse_2602,761559,761559,VN,V,Kazantse,Kazantse,2602,2602,issue,NaN
4,6,1,214663_AV_Dmitriev_1090,214663_AV_Dmitriev_190,214663,214663,AV,AV,Dmitriev,Dmitriev,1090,190,issue,NaN


Далее пошагово обрабатываем все компоненты нового ID: инициалы, фамилию, аффилиации

На каждом этапе фиксируем все параметры кроме одного за счёт фильтра на их равенство между строками в паре и задаем правило выбора между отличающимися значениями свободного параметра

In [ ]:
#здесь работаем только с инициалами
for i in range(len(elid)):
    try:
        assert elid['id1'][i] == elid['id2'][i]
        assert elid['aff1'][i] == elid['aff2'][i]
        assert elid['surname1'][i] == elid['surname2'][i]

        # для инициалов всегда предпочитаем более длинное написание
        if len(elid['init1'][i]) > len(elid['init2'][i]):
            elid['end_res'][i] = elid['val1'][i]
        else:
            elid['end_res'][i] = elid['val2'][i]
    except:
        pass

In [ ]:
print('Количество неназначенных ID сократилось на {}.'.format(len(elid) - elid['end_res'].isnull().sum()))

Количество неназначенных ID сократилось на 763.


In [ ]:
#здесь работаем только с аффилиациями
for i in range(len(elid)):
    try:
        assert elid['id1'][i] == elid['id2'][i]
        assert elid['init1'][i] == elid['init2'][i]
        assert elid['surname1'][i] == elid['surname2'][i]

        # различающиеся аффилиации объединяем и добавляем последовательно в итоговый вывод
        if elid['aff1'][i] != elid['aff2'][i]:
            elid['end_res'][i] = '_'.join([elid['val1'][i], elid['aff2'][i]])

    except:
        pass

In [ ]:
elid.head()

,index,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
0,1,1,483923_OM_Shtompel_322,483923_O_Shtompel_322,483923,483923,OM,O,Shtompel,Shtompel,322,322,issue,483923_OM_Shtompel_322
1,2,1,118648_GI_Osadchaj_1432,118648_G_Osadchaj_1432,118648,118648,GI,G,Osadchaj,Osadchaj,1432,1432,issue,118648_GI_Osadchaj_1432
2,3,1,514063_JUP_Lezhnina_421,514063_JU_Lezhnina_421,514063,514063,JUP,JU,Lezhnina,Lezhnina,421,421,issue,514063_JUP_Lezhnina_421
3,5,1,761559_VN_Kazantse_2602,761559_V_Kazantse_2602,761559,761559,VN,V,Kazantse,Kazantse,2602,2602,issue,761559_VN_Kazantse_2602
4,6,1,214663_AV_Dmitriev_1090,214663_AV_Dmitriev_190,214663,214663,AV,AV,Dmitriev,Dmitriev,1090,190,issue,214663_AV_Dmitriev_1090_190


In [ ]:
print('Количество неназначенных ID сократилось на {}.'.format(len(elid) - elid['end_res'].isnull().sum()))

Количество неназначенных ID сократилось на 7909.


In [ ]:
# к этому шагу 94% кейсов в таблице elid прояснены, прочие сохраняются в отдельный файл для ручной обработки
elid[(elid['end_res'].isnull()) & (elid['id1'] != elid['id2'])].to_excel('not_clear.xlsx')

In [ ]:
elid = elid[elid['end_res'].notnull()].reset_index() #сохраняем решенные кейсы

In [ ]:
#добавляем в словарь финальных результатов в качестве ключа избранную/измененную строку и сравниваемые варианты в качестве значений
for i in tqdm(range(len(elid))):
    if elid['end_res'][i] not in end_dic.keys():
        end_dic[elid['end_res'][i]] = []
    else:
        pass

In [ ]:
for i in range(len(elid)):
    end_dic[elid['end_res'][i]] += [elid['val1'][i], elid['val2'][i]]

## Шаг 2: Обработка авторов с отсутствующим идентификатором

In [ ]:
# для удобства создаем суб-таблицу, содержащую только тех авторов, у которых в id первой строки указан 0
noid = df[df['id1'] == '0'].reset_index(drop = True)
noid['end_res'] = pd.Series()

In [ ]:
noid.head()

,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
0,1,0_EA_Kozhevni_none,0_IA_Kozhevni_none,0,0,EA,IA,Kozhevni,Kozhevni,none,none,issue,NaN
1,1,0_VG_Gojdenko_18476,0_V_Gojdenko_18476,0,0,VG,V,Gojdenko,Gojdenko,18476,18476,issue,NaN
2,1,0_DI_Hismatul_704,0_GI_Hismatul_704,0,0,DI,GI,Hismatul,Hismatul,704,704,issue,NaN
3,1,0_RF_Shajahme_211,0_RR_Shajahme_211,0,0,RF,RR,Shajahme,Shajahme,211,211,issue,NaN
4,1,0_G_Li_867,0_S_Li_867,0,0,G,S,Li,Li,867,867,issue,NaN


Сначала решаем самый простой кейс - случай, где все одинаковое, кроме ID.
Таких наблюдений всего 6

В этих кейсах в итоговый результат добавляется строка с ненулевым ID

In [ ]:
dif_id = noid[(noid['surname1'] == noid['surname2'])
     & (noid['aff1'] == noid['aff2'])
    & (noid['init1'] == noid['init2'])].reset_index()

In [ ]:
for i in range(len(dif_id)):
    dif_id['end_res'][i] = dif_id['val2'][i]
for i in tqdm(range(len(dif_id))):
    if dif_id['end_res'][i] not in end_dic.keys():
        end_dic[dif_id['end_res'][i]] = [dif_id['val1'][i], dif_id['val2'][i]]
    else:
        end_dic[dif_id['end_res'][i]] += [dif_id['val1'][i], dif_id['val2'][i]]

<ipython-input-46-ee90f728d563>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dif_id['end_res'][i] = dif_id['val2'][i]
100%|██████████| 6/6 [00:00<00:00, 10176.23it/s]


In [ ]:
noid = noid[(noid['surname1'] != noid['surname2'])
     | (noid['aff1'] != noid['aff2'])
     | (noid['init1'] != noid['init2'])] #выборка нерешенных кейсов очищается от обработанных наблюдений

Теперь рассматриваем случай, где разнятся инициалы:

In [ ]:
dif_init = noid[(noid['surname1'] == noid['surname2']) &
               (noid['id1'] == noid['id2']) &
               (noid['aff1'] == noid['aff2'])
              ]
dif_init[dif_init['init2'] == 'none']

,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
4306,4,0_EI_Samygin_14461,0_none_Samygin_14461,0,0,EI,none,Samygin,Samygin,14461,14461,issue,NaN
6096,4,0_AV_Chekarev_2541,0_none_Chekarev_2541,0,0,AV,none,Chekarev,Chekarev,2541,2541,issue,NaN
7277,4,0_VF_Neh_4211,0_none_Neh_4211,0,0,VF,none,Neh,Neh,4211,4211,issue,NaN
8114,4,0_V_Neh_4211,0_none_Neh_4211,0,0,V,none,Neh,Neh,4211,4211,issue,NaN
11485,4,0_NV_Grigor'e_none,0_none_Grigor'e_none,0,0,NV,none,Grigor'e,Grigor'e,none,none,issue,NaN
13428,4,0_OB_Nasobin_5,0_none_Nasobin_5,0,0,OB,none,Nasobin,Nasobin,5,5,issue,NaN
17023,4,0_AJU_Grigor'e_none,0_none_Grigor'e_none,0,0,AJU,none,Grigor'e,Grigor'e,none,none,issue,NaN


In [ ]:
for i in [4306,6096,7277,8114,13428]: # часть наблюдений отсеяна вручную из-за неопределенности ввиду отсутствия как инициалов, так и аффилиации
    if dif_init['end_res'][i] not in end_dic.keys():
        end_dic[dif_init['end_res'][i]] = [dif_init['val1'][i], dif_init['val2'][i]]
    else:
        end_dic[dif_init['end_res'][i]] += [dif_init['val1'][i], dif_init['val2'][i]]

Здесь рассматриваю кейсы, где все одинаковое, но аффилиация во второй строке - none

In [ ]:
df_aff = noid[(noid['surname1'] == noid['surname2']) &
     (noid['id1'] == noid['id2']) &
     (noid['init1'] == noid['init2']) &
     (noid['aff1'] != noid['aff2']) & (noid['aff2'] == 'none')
     & (noid['end_res'].isnull())]

df_aff.head()

,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
1915,3,0_LI_Selivano_Gos,0_LI_Selivano_none,0,0,LI,LI,Selivano,Selivano,Gos,none,issue,NaN
3927,4,0_OV_Kersanov_639,0_OV_Kersanov_none,0,0,OV,OV,Kersanov,Kersanov,639,none,issue,NaN
3986,4,0_IJU_Bogachev_804,0_IJU_Bogachev_none,0,0,IJU,IJU,Bogachev,Bogachev,804,none,issue,NaN
4005,4,0_KA_Hamzina_735,0_KA_Hamzina_none,0,0,KA,KA,Hamzina,Hamzina,735,none,issue,NaN
4011,4,0_MN_Garchenk_Sc,0_MN_Garchenk_none,0,0,MN,MN,Garchenk,Garchenk,Sc,none,issue,NaN


In [ ]:
for i in df_aff.index:
    df_aff.end_res[i] = df_aff.val1[i]
df_aff.head()

<ipython-input-55-ec884359cfe8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aff.end_res[i] = df_aff.val1[i]


,diff,val1,val2,id1,id2,init1,init2,surname1,surname2,aff1,aff2,status,end_res
1915,3,0_LI_Selivano_Gos,0_LI_Selivano_none,0,0,LI,LI,Selivano,Selivano,Gos,none,issue,0_LI_Selivano_Gos
3927,4,0_OV_Kersanov_639,0_OV_Kersanov_none,0,0,OV,OV,Kersanov,Kersanov,639,none,issue,0_OV_Kersanov_639
3986,4,0_IJU_Bogachev_804,0_IJU_Bogachev_none,0,0,IJU,IJU,Bogachev,Bogachev,804,none,issue,0_IJU_Bogachev_804
4005,4,0_KA_Hamzina_735,0_KA_Hamzina_none,0,0,KA,KA,Hamzina,Hamzina,735,none,issue,0_KA_Hamzina_735
4011,4,0_MN_Garchenk_Sc,0_MN_Garchenk_none,0,0,MN,MN,Garchenk,Garchenk,Sc,none,issue,0_MN_Garchenk_Sc


In [ ]:
for i in df_aff.index:
    if df_aff['end_res'][i] not in end_dic.keys():
        end_dic[df_aff['end_res'][i]] = [df_aff['val1'][i], df_aff['val2'][i]]
    else:
        end_dic[df_aff['end_res'][i]] += [df_aff['val1'][i], df_aff['val2'][i]]

## Шаг 3: Собираем результаты обработки воедино

In [ ]:
new_ids = pd.DataFrame(columns = ['id', 'change'])
new_ids['id'] = end_dic.keys()
new_ids['id_no'] = pd.Series()
new_ids['init'] = pd.Series()
new_ids['surname'] = pd.Series()
new_ids['aff'] = pd.Series()

for i in tqdm(range(len(new_ids))):
  try:
    vals = new_ids['id'][i].split('_')
    new_ids['id_no'][i] = str(int(vals[0]))
    new_ids['init'][i] = vals[1]
    new_ids['surname'][i]= vals[2]
    new_ids['aff'][i]= vals[3:]
  except:
    print(new_ids.id[i])

In [ ]:
# создаем отдельный параметр длины списка аффилиаций
new_ids['num_af'] = [len(i) for i in new_ids['aff']]
# сортируем список аффилиаций, в случае если он содержит множественные значения
new_ids['aff'] = [sorted(i) for i in new_ids['aff']]
ch = new_ids[new_ids['num_af'] >1]

In [ ]:
for i in tqdm(ch.index):
  # соединяем множественные ID, если они не none, в противном случае берем не none значение из списка
    if 'none' in new_ids['aff'][i]:
        new_ids.change[i] = '_'.join([new_ids.id_no[i],
                                    new_ids.init[i],
                                    new_ids.surname[i],
                                    new_ids.aff[i][0]])
    else:
        new_ids.change[i] = '_'.join([new_ids.id_no[i],
                                    new_ids.init[i],
                                    new_ids.surname[i],
                                    '_'.join(new_ids.aff[i])])

## Шаг 4: Изменяем рабочий файл с данными

In [ ]:
work_df = pd.read_parquet('work_df_2.parquet')

In [ ]:
work_df.head(50)

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,url,ref,new_id_1,new_id_2,new_id_3,new_id_4,new_id_5,new_id_6,new_id_7,new_id_8
0,15485609,4,0,[7113],Кийков,А.В.,[Белгородский Государственный Технологический ...,Kijkov,A.V.,['Belgorod State Technological University name...,...,none,"Кийков, А.В. СОСТОЯНИЕ И ТЕНДЕНЦИИ СЕТЕВОГО ВЗ...",0_AV_Kijkov_7113,0_IH_Haziev_364,N/A,N/A,N/A,N/A,N/A,N/A
1,15488335,4,675896,[none],Салагаев,А.,[none],Salagaev,A.,['none'],...,none,"Салагаев, А. КУЛЬТУРНАЯ И РЕЛИГИОЗНАЯ СИТУАЦИЯ...",675896_AL_Salagaev_296_none,692631_SA_Sergeev_296_none,755224_LV_Luchshev_296_none,N/A,N/A,N/A,N/A,N/A
2,15488853,4,820606,[340],Семченко,И.В.,[Белгородский государственный национальный исс...,Semchenko,I.V.,['none'],...,none,"Семченко, И.В. ПРОБЛЕМЫ СОЦИАЛЬНОЙ АДАПТАЦИИ Г...",820606_IV_Semchenk_340,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,15500786,4,77952,[210],Хагуров,Т.А.,[Кубанский государственный университет ],Hagurov,T.A.,['none'],...,none,"Хагуров, Т.А. Современная культура и воспитани...",77952_TA_Hagurov_1074_210,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,15501422,4,746025,[2541],Елишев,С.О.,[Московский государственный университет им. М....,Elishev,S.O.,['none'],...,none,"Елишев, С.О. Формирование ценностных ориентаци...",746025_SO_Elishev_2541,N/A,N/A,N/A,N/A,N/A,N/A,N/A
5,15502590,4,726748,[322],Барков,Ф.А.,[Южный федеральный университет ],Barkov,F.A.,['The Institute Retraining and Improvement of ...,...,none,"Барков, Ф.А. ТРАНСФОРМАЦИЯ СОЦИАЛЬНЫХ ПРАКТИК ...",726748_FA_Barkov_1432_322,170040_AV_Serikov_1432_322,N/A,N/A,N/A,N/A,N/A,N/A
6,15502840,4,529532,[634],Хрупин,С.И.,[Адыгейский государственный университет ],Hrupin,S.I.,['none'],...,none,"Хрупин, С.И. Ценности религиозной семьи в созн...",529532_SI_Hrupin_634_none,N/A,N/A,N/A,N/A,N/A,N/A,N/A
7,15507187,4,341669,[919],Килимова,Л.В.,[Юго-Западный государственный университет ],Kilimova,L.V.,['SWSU'],...,none,"Килимова, Л.В. СУЩНОСТЬ И СТРУКТУРА РОССИЙСКОГ...",341669_LV_Kilimova_919,396797_OO_Nishnian_919,616741_AV_Sapronov_919_none,N/A,N/A,N/A,N/A,N/A
8,15510295,4,251611,[none],Савченко,Е.,[none],Savchenko,E.,['none'],...,none,Савченко Е. Юбиляра поздравляют / Савченко Е. ...,251611_ES_Savchenk_none,N/A,N/A,N/A,N/A,N/A,N/A,N/A
9,15511500,4,0,[735],Ермакова,М.Г.,[Казанский национальный исследовательский техн...,Ermakova,M.G.,['Kazan National Research Technical University...,...,none,"Ермакова, М.Г. Дискриминация женщин на рынке т...",0_MG_Ermakova_735,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [ ]:
ids = [i for i in work_df.columns if 'new_id' in i]

In [ ]:
# для всех новых ID, если они встречаются в нашем списке решенных кейсов, заменяем их текущее значение на новое назначенное
for col in ids:
    for i in tqdm(range(len(work_df))):
        try:
            assert work_df[col][i] in all_ids

            for l in end_dic.keys():
                if work_df[col][i] in end_dic[l]:
                    work_df[col][i] = l

        except:
            pass